# Backtesting of VaR Models

In [ ]:
%%capture
# Installing Yfinance package used to download data
!pip install yfinance --upgrade --no-cache-dir
import yfinance as yf

In [ ]:
%%capture
# vargan_model.ipynb does the pretraining of the GAN, we need to import some things for additional training and backtesting.
!pip install import-ipynb
import import_ipynb

In [ ]:
# If running on Google colab, to give acces to the drive
from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/My Drive/Colab Notebooks

In [ ]:
%%capture

import tensorflow as tf
#import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as st 
#import dataPrepFunc as dp
from tensorflow import keras 
from tensorflow.keras import layers
from vargan_model import WGAN, mean_tot, std_tot, to_stock_M, get_returns, generator_loss, discriminator_loss, varGan_sim

In [ ]:
# Prepare the backtesting data as was done for the data used for pretraining the model
data_back_test = yf.download("HM-B.ST EKTA-B.ST TEL2-B.ST SEB-A.ST INVE-B.ST", start="2009-12-21", end="2020-01-01")
data_back_test = data_back_test['Adj Close']
# Impute with previous valid value if there is missing data 
data_test=data_back_test.fillna(method='ffill')
# Make into numpy array
data_test = data_test.to_numpy()
data_test = get_returns(data_test)
data_test_gan = to_stock_M(data_test, 1).astype("float32")

## 1. Unconditional VaR Estimation with GAN

In [ ]:
# load the pretrained GAN
generator_uncon = tf.keras.models.load_model('generator_pretrained.h5')
discriminator_uncon = tf.keras.models.load_model('discriminator_pretrained.h5')

In [ ]:
# optimezers to use, for additional training
generator_optimizer = keras.optimizers.Adam(
    learning_rate=0.00002, beta_1=0.5, beta_2=0.9
)

discriminator_optimizer = keras.optimizers.Adam(
    learning_rate=0.00002, beta_1=0.5, beta_2=0.9
)

k=5
noise_dim = 2*k

In [ ]:
# Function to calculate VaR estimates over a test period 
# we train it for additional 100 epoch on  the 252 most recent days
# and use this estimate for 5 days

def VaR_est(test_data, generator=generator_uncon, discriminator=discriminator_uncon, update_freq = 5, additional_epochs = 100, b_window = 252):
    
    # list to store var estimates over testing period
    
    var_estimates =[]
    
    nbr_days_to_predict = test_data.shape[0]-b_window
    nbr_estimates = (nbr_days_to_predict)/update_freq
    nbr_estimates = int(np.ceil(nbr_estimates))
    
    for i in range(nbr_estimates):
        start_recent=i*update_freq
        end_recent=start_recent + b_window
        data_recent = test_data[start_recent:end_recent,:,:]
        # Now we train the generator for additional epochs
        discriminator_up = discriminator
        generator_up = generator
        wgan_up = WGAN(
        discriminator = discriminator_up,
        generator = generator_up,
        latent_dim=noise_dim,
        disc_extra_steps=5,)
        # Compile 
        wgan_up.compile(
        d_optimizer=discriminator_optimizer,
        g_optimizer=generator_optimizer,
        g_loss_fn=generator_loss,
        d_loss_fn=discriminator_loss)
        # Continue training 
        varGan_up = wgan_up.fit(data_recent, batch_size = 36, epochs = additional_epochs, verbose = 0)
        # Use updated model to estimate VaR 
        var_val = varGan_sim(generator=generator_up)
        # The estimate is the same for update_freq nbr of days
        nbr_new_estimates = np.min([update_freq, nbr_days_to_predict-start_recent])
        for a in range(nbr_new_estimates):
            var_estimates.append(var_val)
        print(f'VaR estimates for days {i*update_freq +1} --> {(i+1)*update_freq} done') 
            
    return var_estimates            

In [ ]:
# VaR estimates for the test data
estimated_var = VaR_est(data_test_gan, update_freq= 5, additional_epochs = 100)

In [ ]:
VaR_uncon = np.array(estimated_var)

## 2. Variance Covariance Method Based on 252 Most Recent Returns  

In [ ]:
def vcv_Var(data, k = 5, alfa = 0.05, obs_b = 252):
    data_length = data.shape[0]
    p_w = 1/k
    portfolio_weights = p_w * np.ones((k,1))
    # List to store var values in
    VaR_list = []
    for i in range(0, data_length-obs_b):
        start = i
        end = obs_b + i
        est_data = data[start:end,]
        # Portfolio return
        mu_hat = np.mean(est_data, axis = 0)
        p_mu = np.matmul(portfolio_weights.transpose(), mu_hat)
        # Sample covariance matrix
        cov_hat = np.cov(est_data, rowvar = False)
        # Calculate portfolio variance
        a = np.matmul(portfolio_weights.transpose(), cov_hat)
        p_variance = np.matmul(a, portfolio_weights)
        # z-score 
        z = st.norm.ppf(1-alfa)
        # VaR
        VaR = -(p_mu - ( z*np.sqrt(p_variance)))
        VaR_list.append(VaR)   
    return VaR_list

In [ ]:
var_list = vcv_Var(data_test)

In [ ]:
VaR_vcv = np.array(var_list).reshape((len(var_list)))

## 3. Plotting VaR Estimates Against the Acctual Returns 

For a good model we expect about 5% of the returns to be below the returns given by the model.

In [ ]:
# The acctual returns, for backtesting, note first 252 used for estimation
start_back_test = data_test.shape[0] - 252
actual_returns = data_test[-start_back_test:,:]

# period for back test
b_period =data_back_test.index
b_period = b_period.to_numpy()
b_period = b_period[253:]

In [ ]:
# Calculate acctual returns for the equaly weighted portfolio
def portfolio_returns(data, k=5):
  days = data.shape[0]
  p_w = 1/k
  portfolio_weights = p_w * np.ones((k,1))
  p_returns = []
  for row in range(days):
    returns = data[row,:]
    p_r = np.matmul(portfolio_weights.transpose(), returns)
    p_returns.append(p_r)
  p_returns = np.array(p_returns).reshape((days))
  return p_returns     

In [ ]:
actual_returns_p = portfolio_returns(actual_returns)

In [ ]:
#  VaR estimates of WGAN-GP model against accual returns
plt.plot_date(b_period, VaR_uncon, ls="solid", marker ="None", label = "Unconditional GAN (model 2)")
plt.plot_date(b_period, actual_returns_p, label = "Actual portfolio returns", markersize=1, color = "green")
plt.legend(loc="upper left", fontsize="small")
plt.xlabel("Date")
plt.ylabel("Return")

In [ ]:
# VCV model VS acctual returns 
plt.plot_date(b_period, -VaR_vcv, ls="solid", marker ="None", label = "Variance covariance (model 6)")
plt.plot_date(b_period, actual_returns_p, label = "Actual portfolio returns", markersize=1, color = "green")
plt.legend(loc="upper left", fontsize="small")
plt.xlabel("Date")
plt.ylabel("Return")